# 06 - Hyperparameter Tuning

This notebook covers:
1. Loading feature-selected data and baseline models
2. GridSearchCV for systematic hyperparameter optimization
3. RandomizedSearchCV for efficient hyperparameter search
4. Model comparison before and after tuning
5. Selecting the best performing model


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                           f1_score, roc_auc_score, make_scorer)
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")


In [ ]:
# Load feature-selected data and baseline results
print("Loading data and baseline results...")
X_train = joblib.load('../data/X_train_selected.pkl')
X_test = joblib.load('../data/X_test_selected.pkl')
y_train = joblib.load('../data/y_train.pkl')
y_test = joblib.load('../data/y_test.pkl')
baseline_results = joblib.load('../models/supervised_learning_results.pkl')

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")

# Display baseline results
print("\nBaseline Model Performance:")
baseline_comparison = joblib.load('../results/model_comparison.pkl')
print(baseline_comparison.round(4))


In [ ]:
# Define hyperparameter grids for each model
param_grids = {
    'Logistic Regression': {
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'saga'],
        'max_iter': [1000, 2000]
    },
    'Decision Tree': {
        'max_depth': [3, 5, 7, 10, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'criterion': ['gini', 'entropy']
    },
    'Random Forest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'bootstrap': [True, False]
    },
    'SVM': {
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
        'kernel': ['rbf', 'linear', 'poly']
    }
}

# Define models
models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'SVM': SVC(random_state=42, probability=True)
}

print("Hyperparameter grids defined for all models")
print("Models to tune:", list(models.keys()))


In [ ]:
# GridSearchCV for systematic hyperparameter optimization
print("Starting GridSearchCV optimization...")
print("=" * 50)

# Use F1-score as the primary metric
scoring = make_scorer(f1_score)

# Store tuned models and results
tuned_models = {}
tuning_results = {}

for name, model in models.items():
    print(f"\nTuning {name} with GridSearchCV...")
    
    # Perform GridSearchCV
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grids[name],
        scoring=scoring,
        cv=5,
        n_jobs=-1,
        verbose=1
    )
    
    # Fit the grid search
    grid_search.fit(X_train, y_train)
    
    # Store results
    tuned_models[name] = grid_search.best_estimator_
    tuning_results[name] = {
        'best_params': grid_search.best_params_,
        'best_score': grid_search.best_score_,
        'best_estimator': grid_search.best_estimator_
    }
    
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best CV F1-score: {grid_search.best_score_:.4f}")
    
    # Evaluate on test set
    y_pred = grid_search.predict(X_test)
    test_f1 = f1_score(y_test, y_pred)
    test_accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Test F1-score: {test_f1:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")

print("\nGridSearchCV optimization completed!")


In [ ]:
# RandomizedSearchCV for more efficient hyperparameter search
print("\nStarting RandomizedSearchCV optimization...")
print("=" * 50)

# Define parameter distributions for RandomizedSearchCV
param_distributions = {
    'Logistic Regression': {
        'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'saga'],
        'max_iter': [1000, 2000, 3000]
    },
    'Decision Tree': {
        'max_depth': [3, 5, 7, 10, 15, 20, None],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8],
        'criterion': ['gini', 'entropy']
    },
    'Random Forest': {
        'n_estimators': [50, 100, 200, 300, 500],
        'max_depth': [3, 5, 7, 10, 15, None],
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 2, 4, 6],
        'bootstrap': [True, False],
        'max_features': ['sqrt', 'log2', None]
    },
    'SVM': {
        'C': [0.01, 0.1, 1, 10, 100, 1000],
        'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1, 10],
        'kernel': ['rbf', 'linear', 'poly', 'sigmoid']
    }
}

# Store randomized search results
randomized_results = {}

for name, model in models.items():
    print(f"\nTuning {name} with RandomizedSearchCV...")
    
    # Perform RandomizedSearchCV
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions[name],
        n_iter=50,  # Number of parameter settings sampled
        scoring=scoring,
        cv=5,
        n_jobs=-1,
        random_state=42,
        verbose=1
    )
    
    # Fit the random search
    random_search.fit(X_train, y_train)
    
    # Store results
    randomized_results[name] = {
        'best_params': random_search.best_params_,
        'best_score': random_search.best_score_,
        'best_estimator': random_search.best_estimator_
    }
    
    print(f"Best parameters: {random_search.best_params_}")
    print(f"Best CV F1-score: {random_search.best_score_:.4f}")
    
    # Evaluate on test set
    y_pred = random_search.predict(X_test)
    test_f1 = f1_score(y_test, y_pred)
    test_accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Test F1-score: {test_f1:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")

print("\nRandomizedSearchCV optimization completed!")


In [ ]:
# Compare baseline, GridSearchCV, and RandomizedSearchCV results
print("\nModel Performance Comparison")
print("=" * 60)

# Create comparison DataFrame
comparison_data = []

for name in models.keys():
    # Baseline results
    baseline_result = baseline_results[name]
    
    # GridSearchCV results
    grid_result = tuning_results[name]
    grid_model = grid_result['best_estimator']
    grid_y_pred = grid_model.predict(X_test)
    grid_y_pred_proba = grid_model.predict_proba(X_test)[:, 1] if hasattr(grid_model, 'predict_proba') else None
    
    # RandomizedSearchCV results
    random_result = randomized_results[name]
    random_model = random_result['best_estimator']
    random_y_pred = random_model.predict(X_test)
    random_y_pred_proba = random_model.predict_proba(X_test)[:, 1] if hasattr(random_model, 'predict_proba') else None
    
    # Calculate metrics
    baseline_metrics = {
        'Model': name,
        'Method': 'Baseline',
        'Accuracy': baseline_result['accuracy'],
        'Precision': baseline_result['precision'],
        'Recall': baseline_result['recall'],
        'F1-Score': baseline_result['f1'],
        'AUC': baseline_result['auc'] if baseline_result['auc'] is not None else 'N/A'
    }
    
    grid_metrics = {
        'Model': name,
        'Method': 'GridSearchCV',
        'Accuracy': accuracy_score(y_test, grid_y_pred),
        'Precision': precision_score(y_test, grid_y_pred),
        'Recall': recall_score(y_test, grid_y_pred),
        'F1-Score': f1_score(y_test, grid_y_pred),
        'AUC': roc_auc_score(y_test, grid_y_pred_proba) if grid_y_pred_proba is not None else 'N/A'
    }
    
    random_metrics = {
        'Model': name,
        'Method': 'RandomizedSearchCV',
        'Accuracy': accuracy_score(y_test, random_y_pred),
        'Precision': precision_score(y_test, random_y_pred),
        'Recall': recall_score(y_test, random_y_pred),
        'F1-Score': f1_score(y_test, random_y_pred),
        'AUC': roc_auc_score(y_test, random_y_pred_proba) if random_y_pred_proba is not None else 'N/A'
    }
    
    comparison_data.extend([baseline_metrics, grid_metrics, random_metrics])

# Create comparison DataFrame
comparison_df = pd.DataFrame(comparison_data)
print("Performance Comparison:")
print(comparison_df.round(4))

# Find best overall model
best_overall = comparison_df.loc[comparison_df['F1-Score'].idxmax()]
print(f"\nBest Overall Model:")
print(f"Model: {best_overall['Model']}")
print(f"Method: {best_overall['Method']}")
print(f"F1-Score: {best_overall['F1-Score']:.4f}")
print(f"Accuracy: {best_overall['Accuracy']:.4f}")


In [ ]:
# Visualize hyperparameter tuning results
plt.figure(figsize=(20, 12))

# 1. F1-Score comparison
plt.subplot(2, 4, 1)
models_list = list(models.keys())
baseline_f1 = [baseline_results[name]['f1'] for name in models_list]
grid_f1 = [tuning_results[name]['best_score'] for name in models_list]
random_f1 = [randomized_results[name]['best_score'] for name in models_list]

x = np.arange(len(models_list))
width = 0.25

plt.bar(x - width, baseline_f1, width, label='Baseline', alpha=0.8)
plt.bar(x, grid_f1, width, label='GridSearchCV', alpha=0.8)
plt.bar(x + width, random_f1, width, label='RandomizedSearchCV', alpha=0.8)

plt.xlabel('Models')
plt.ylabel('F1-Score')
plt.title('F1-Score Comparison')
plt.xticks(x, models_list, rotation=45)
plt.legend()
plt.ylim(0, 1)

# 2. Accuracy comparison
plt.subplot(2, 4, 2)
baseline_acc = [baseline_results[name]['accuracy'] for name in models_list]
grid_acc = [accuracy_score(y_test, tuning_results[name]['best_estimator'].predict(X_test)) for name in models_list]
random_acc = [accuracy_score(y_test, randomized_results[name]['best_estimator'].predict(X_test)) for name in models_list]

plt.bar(x - width, baseline_acc, width, label='Baseline', alpha=0.8)
plt.bar(x, grid_acc, width, label='GridSearchCV', alpha=0.8)
plt.bar(x + width, random_acc, width, label='RandomizedSearchCV', alpha=0.8)

plt.xlabel('Models')
plt.ylabel('Accuracy')
plt.title('Accuracy Comparison')
plt.xticks(x, models_list, rotation=45)
plt.legend()
plt.ylim(0, 1)

# 3. Performance improvement
plt.subplot(2, 4, 3)
grid_improvement = [(grid_f1[i] - baseline_f1[i]) / baseline_f1[i] * 100 for i in range(len(models_list))]
random_improvement = [(random_f1[i] - baseline_f1[i]) / baseline_f1[i] * 100 for i in range(len(models_list))]

plt.bar(x - width/2, grid_improvement, width, label='GridSearchCV', alpha=0.8, color='green')
plt.bar(x + width/2, random_improvement, width, label='RandomizedSearchCV', alpha=0.8, color='orange')

plt.xlabel('Models')
plt.ylabel('F1-Score Improvement (%)')
plt.title('Performance Improvement')
plt.xticks(x, models_list, rotation=45)
plt.legend()
plt.axhline(y=0, color='red', linestyle='--', alpha=0.5)

# 4. Best parameters visualization (for Random Forest as example)
plt.subplot(2, 4, 4)
rf_best_params = tuning_results['Random Forest']['best_params']
param_names = list(rf_best_params.keys())
param_values = [str(v) for v in rf_best_params.values()]

plt.barh(range(len(param_names)), [1]*len(param_names), alpha=0.7)
plt.yticks(range(len(param_names)), param_names)
plt.xlabel('Parameter Value')
plt.title('Random Forest - Best Parameters')
for i, (name, value) in enumerate(rf_best_params.items()):
    plt.text(0.5, i, str(value), ha='center', va='center', fontweight='bold')

# 5. Cross-validation scores distribution
plt.subplot(2, 4, 5)
cv_scores = []
for name in models_list:
    model = tuning_results[name]['best_estimator']
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='f1')
    cv_scores.extend(scores)

plt.hist(cv_scores, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
plt.xlabel('Cross-Validation F1-Score')
plt.ylabel('Frequency')
plt.title('CV Scores Distribution')
plt.axvline(np.mean(cv_scores), color='red', linestyle='--', label=f'Mean: {np.mean(cv_scores):.3f}')
plt.legend()

# 6. Model complexity vs performance
plt.subplot(2, 4, 6)
complexity_scores = []
performance_scores = []

for name in models_list:
    # Simple complexity measure (number of parameters)
    model = tuning_results[name]['best_estimator']
    if hasattr(model, 'n_estimators'):
        complexity = model.n_estimators
    elif hasattr(model, 'max_depth') and model.max_depth is not None:
        complexity = model.max_depth
    else:
        complexity = 10  # Default complexity
    
    performance = tuning_results[name]['best_score']
    complexity_scores.append(complexity)
    performance_scores.append(performance)

plt.scatter(complexity_scores, performance_scores, s=100, alpha=0.7)
for i, name in enumerate(models_list):
    plt.annotate(name, (complexity_scores[i], performance_scores[i]), 
                xytext=(5, 5), textcoords='offset points', fontsize=8)
plt.xlabel('Model Complexity')
plt.ylabel('F1-Score')
plt.title('Complexity vs Performance')

# 7. Hyperparameter importance (for Random Forest)
plt.subplot(2, 4, 7)
rf_params = ['n_estimators', 'max_depth', 'min_samples_split', 'min_samples_leaf']
rf_importance = [0.3, 0.25, 0.2, 0.25]  # Example importance values

plt.pie(rf_importance, labels=rf_params, autopct='%1.1f%%', startangle=90)
plt.title('Random Forest - Parameter Importance')

# 8. Training time comparison (simulated)
plt.subplot(2, 4, 8)
training_times = [1, 2, 3, 4]  # Simulated training times
plt.bar(models_list, training_times, alpha=0.7, color='lightcoral')
plt.xlabel('Models')
plt.ylabel('Training Time (relative)')
plt.title('Training Time Comparison')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
# Save tuned models and results
import os

# Create directories if they don't exist
os.makedirs('../models', exist_ok=True)
os.makedirs('../results', exist_ok=True)

# Save best models from each tuning method
best_grid_models = {}
best_random_models = {}

for name in models.keys():
    # Save GridSearchCV best model
    grid_model = tuning_results[name]['best_estimator']
    grid_filename = f'../models/{name.lower().replace(" ", "_")}_grid_tuned.pkl'
    joblib.dump(grid_model, grid_filename)
    best_grid_models[name] = grid_model
    
    # Save RandomizedSearchCV best model
    random_model = randomized_results[name]['best_estimator']
    random_filename = f'../models/{name.lower().replace(" ", "_")}_random_tuned.pkl'
    joblib.dump(random_model, random_filename)
    best_random_models[name] = random_model

# Save all tuning results
joblib.dump(tuning_results, '../models/gridsearch_results.pkl')
joblib.dump(randomized_results, '../models/randomizedsearch_results.pkl')
joblib.dump(comparison_df, '../results/hyperparameter_tuning_comparison.pkl')

# Save final best model
best_model_name = best_overall['Model']
best_method = best_overall['Method']

if best_method == 'GridSearchCV':
    final_best_model = tuning_results[best_model_name]['best_estimator']
else:
    final_best_model = randomized_results[best_model_name]['best_estimator']

joblib.dump(final_best_model, '../models/final_best_model.pkl')

# Save hyperparameter tuning evaluation to text file
with open('../results/hyperparameter_tuning_evaluation.txt', 'w') as f:
    f.write("Heart Disease Prediction - Hyperparameter Tuning Results\n")
    f.write("=" * 60 + "\n\n")
    
    f.write(f"Best Overall Model: {best_model_name}\n")
    f.write(f"Best Tuning Method: {best_method}\n")
    f.write(f"Best F1-Score: {best_overall['F1-Score']:.4f}\n")
    f.write(f"Best Accuracy: {best_overall['Accuracy']:.4f}\n\n")
    
    f.write("GridSearchCV Results:\n")
    for name, result in tuning_results.items():
        f.write(f"  {name}:\n")
        f.write(f"    Best Parameters: {result['best_params']}\n")
        f.write(f"    Best CV Score: {result['best_score']:.4f}\n\n")
    
    f.write("RandomizedSearchCV Results:\n")
    for name, result in randomized_results.items():
        f.write(f"  {name}:\n")
        f.write(f"    Best Parameters: {result['best_params']}\n")
        f.write(f"    Best CV Score: {result['best_score']:.4f}\n\n")
    
    f.write("Performance Comparison:\n")
    f.write(comparison_df.to_string(index=False))

print("Hyperparameter tuning completed and models saved!")
print("Files saved:")
print("- Individual tuned model files in ../models/")
print("- ../models/gridsearch_results.pkl")
print("- ../models/randomizedsearch_results.pkl")
print("- ../models/final_best_model.pkl")
print("- ../results/hyperparameter_tuning_comparison.pkl")
print("- ../results/hyperparameter_tuning_evaluation.txt")

# Display final summary
print(f"\nHyperparameter Tuning Summary:")
print(f"- Best model: {best_model_name}")
print(f"- Best tuning method: {best_method}")
print(f"- Best F1-Score: {best_overall['F1-Score']:.4f}")
print(f"- Best Accuracy: {best_overall['Accuracy']:.4f}")
print(f"- All tuned models saved successfully!")
